<a href="https://colab.research.google.com/github/simranarora1996/US-income-data-sampling/blob/main/Monte_Carlo_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Missing Data Imputation (Adult Census 1994) — MICE vs MH vs Gibbs

**Research question:** How do different imputation methods affect income classification when key continuous predictors contain missing values?

**Methods compared**
- Frequentist baseline: Mean imputation
- MICE (Iterative imputation)
- Metropolis–Hastings (from scratch)
- Gibbs sampling (from scratch)

**Evaluation**
- Downstream model: Logistic Regression
- Predictive metric: AUC (primary), plus Accuracy / F1 / LogLoss
- Imputation quality: MAE / RMSE on artificially masked test entries
- Missingness levels: 5%, 10%, 20% (MCAR)



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, log_loss, confusion_matrix
from sklearn.impute import SimpleImputer

from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.impute import IterativeImputer

from sklearn.metrics import mean_absolute_error, mean_squared_error

pd.set_option("display.max_columns", 50)
np.random.seed(42)



## 1) Load data and select predictors

We use three continuous predictors (Option A):
- `age`
- `hours.per.week`
- `education.num`

Target:
- `income` converted to 0/1 (<=50K vs >50K)


In [ ]:
DATA_PATH = "adult.csv"  # adjust if needed

df = pd.read_csv(DATA_PATH)
print("Dataset shape:", df.shape)
display(df.head())




FileNotFoundError: [Errno 2] No such file or directory: 'adult.csv'

In [ ]:
target_col = "income"
continuous_cols = ["age", "hours.per.week", "education.num"]

y_full = df[target_col].astype(str).str.strip().eq(">50K").astype(int)
X_full = df[continuous_cols].copy()

print("Target counts (0=<=50K, 1=>50K):")
print(y_full.value_counts())
display(X_full.describe().T)




## 2) Stratified sample (N=1000) and baseline logistic regression

We sample N=1000 with class proportions similar to the full dataset.


In [ ]:
def stratified_sample(df, y, cols, N=1000, seed=42):
    tmp = df.copy()
    tmp["_y"] = y.values

    p1 = tmp["_y"].mean()
    n1 = int(round(N * p1))
    n0 = N - n1

    df0 = tmp[tmp["_y"] == 0].sample(n=n0, random_state=seed)
    df1 = tmp[tmp["_y"] == 1].sample(n=n1, random_state=seed)

    out = pd.concat([df0, df1]).sample(frac=1, random_state=seed).reset_index(drop=True)
    y_out = out[target_col].astype(str).str.strip().eq(">50K").astype(int)
    X_out = out[cols].copy()
    return X_out, y_out

X_s, y_s = stratified_sample(df, y_full, continuous_cols, N=1000, seed=42)

print("Sample shape:", X_s.shape)
print("Sample target distribution:")
print(y_s.value_counts(normalize=True).rename("proportion"))




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_s, y_s, test_size=0.25, random_state=42, stratify=y_s
)

baseline_model = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=2000, solver="lbfgs"))
])

baseline_model.fit(X_train, y_train)
proba = baseline_model.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

baseline_metrics = {
    "Accuracy": accuracy_score(y_test, pred),
    "AUC": roc_auc_score(y_test, proba),
    "F1": f1_score(y_test, pred),
    "LogLoss": log_loss(y_test, proba),
}
display(pd.DataFrame([baseline_metrics]).T.rename(columns={0: "Baseline (no missingness)"}))
print("Confusion matrix [[TN FP],[FN TP]]:")
print(confusion_matrix(y_test, pred))





## 3) Missingness injection (MCAR)

We inject missingness into each predictor independently at:
- 5%, 10%, 20%

To keep comparisons fair, all methods use the same masks for a given rate.


In [ ]:
missing_rates = [0.05, 0.10, 0.20]

def make_mcar_mask(X: pd.DataFrame, missing_rate: float, seed: int) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    mask = pd.DataFrame(False, index=X.index, columns=X.columns)

    n = len(X)
    k = int(round(missing_rate * n))
    for col in X.columns:
        idx = rng.choice(X.index.to_numpy(), size=k, replace=False)
        mask.loc[idx, col] = True
    return mask

def apply_mask(X: pd.DataFrame, mask: pd.DataFrame) -> pd.DataFrame:
    X_m = X.copy()
    for col in X.columns:
        X_m.loc[mask[col], col] = np.nan
    return X_m

def missing_report(X_m: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame({
        "missing_count": X_m.isna().sum(),
        "missing_pct": (X_m.isna().mean() * 100).round(2)
    })

data_by_rate = {}

for r in missing_rates:
    seed_r = int(r * 10_000) + 42
    mask_tr = make_mcar_mask(X_train, r, seed=seed_r)
    mask_te = make_mcar_mask(X_test,  r, seed=seed_r + 1)

    Xtr_m = apply_mask(X_train, mask_tr)
    Xte_m = apply_mask(X_test,  mask_te)

    data_by_rate[r] = {
        "X_train_true": X_train.copy(),
        "X_test_true": X_test.copy(),
        "y_train": y_train.copy(),
        "y_test": y_test.copy(),
        "mask_train": mask_tr,
        "mask_test": mask_te,
        "X_train_miss": Xtr_m,
        "X_test_miss": Xte_m
    }

    print(f"\nMissing rate = {int(r*100)}%")
    display(missing_report(Xtr_m))



## 4) Common evaluation + imputation error functions

- Logistic regression is trained on imputed training data and evaluated on imputed test data.
- Imputation error is computed only on the artificially masked entries.




In [ ]:
def evaluate_logreg(X_train_imp, y_train, X_test_imp, y_test):
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(max_iter=2000, solver="lbfgs"))
    ])
    model.fit(X_train_imp, y_train)

    proba = model.predict_proba(X_test_imp)[:, 1]
    pred = (proba >= 0.5).astype(int)

    return {
        "Accuracy": accuracy_score(y_test, pred),
        "AUC": roc_auc_score(y_test, proba),
        "F1": f1_score(y_test, pred),
        "LogLoss": log_loss(y_test, proba),
    }

def imputation_error(true_df, imp_df, mask_df):
    rows = []
    for col in true_df.columns:
        true_vals = true_df.loc[mask_df[col], col].to_numpy()
        imp_vals  = imp_df.loc[mask_df[col], col].to_numpy()
        mae = mean_absolute_error(true_vals, imp_vals)
        rmse = np.sqrt(mean_squared_error(true_vals, imp_vals))
        rows.append({"feature": col, "MAE": mae, "RMSE": rmse})
    return pd.DataFrame(rows).set_index("feature")



## 5) Frequentist baseline: Mean imputation


In [ ]:
def mean_impute(X_train_miss, X_test_miss):
    imp = SimpleImputer(strategy="mean")
    Xtr = pd.DataFrame(imp.fit_transform(X_train_miss), columns=X_train_miss.columns, index=X_train_miss.index)
    Xte = pd.DataFrame(imp.transform(X_test_miss), columns=X_test_miss.columns, index=X_test_miss.index)
    return Xtr, Xte

mean_results = []
mean_errors = {}

for r in missing_rates:
    pack = data_by_rate[r]
    Xtr_imp, Xte_imp = mean_impute(pack["X_train_miss"], pack["X_test_miss"])

    met = evaluate_logreg(Xtr_imp, pack["y_train"], Xte_imp, pack["y_test"])
    mean_results.append({"MissingRate": int(r * 100), **met})

    err_te = imputation_error(pack["X_test_true"], Xte_imp, pack["mask_test"])
    mean_errors[r] = {"test": err_te}

mean_results_df = pd.DataFrame(mean_results).set_index("MissingRate").sort_index()
display(mean_results_df)



## 6) MICE (Iterative imputation)


In [ ]:
mice_results = []
mice_impute_errors = {}

for r in missing_rates:
    pack = data_by_rate[r]

    imp = IterativeImputer(
        random_state=42,
        sample_posterior=True,
        max_iter=15,
        initial_strategy="mean"
    )

    Xtr_imp = pd.DataFrame(
        imp.fit_transform(pack["X_train_miss"]),
        columns=pack["X_train_miss"].columns,
        index=pack["X_train_miss"].index
    )
    Xte_imp = pd.DataFrame(
        imp.transform(pack["X_test_miss"]),
        columns=pack["X_test_miss"].columns,
        index=pack["X_test_miss"].index
    )

    met = evaluate_logreg(Xtr_imp, pack["y_train"], Xte_imp, pack["y_test"])
    mice_results.append({"MissingRate": int(r * 100), **met})

    err_te = imputation_error(pack["X_test_true"], Xte_imp, pack["mask_test"])
    mice_impute_errors[r] = {"test": err_te}

mice_results_df = pd.DataFrame(mice_results).set_index("MissingRate").sort_index()
display(mice_results_df)



## 7) Metropolis–Hastings imputation (from scratch)

We use conditional Gaussian regression models:
\[
x_j \mid x_{-j} \sim \mathcal{N}(\beta_0 + \beta^T x_{-j}, \sigma^2)
\]
Missing entries are sampled via random-walk MH, then replaced by the post-burn-in mean.


In [ ]:
def fit_conditional_models(X_miss: pd.DataFrame):
    models = {}
    cols = list(X_miss.columns)

    for col in cols:
        other_cols = [c for c in cols if c != col]

        ok = X_miss[col].notna()
        for oc in other_cols:
            ok &= X_miss[oc].notna()

        X_other = X_miss.loc[ok, other_cols].to_numpy()
        y = X_miss.loc[ok, col].to_numpy()

        if len(y) < 20:
            mu = float(X_miss[col].mean())
            beta = np.array([mu] + [0.0] * len(other_cols))
            sigma = float(X_miss[col].std() + 1e-6)
            models[col] = (beta, sigma, other_cols)
            continue

        X_design = np.c_[np.ones(len(X_other)), X_other]
        lam = 1e-6
        XtX = X_design.T @ X_design + lam * np.eye(X_design.shape[1])
        Xty = X_design.T @ y
        beta = np.linalg.solve(XtX, Xty)

        resid = y - X_design @ beta
        sigma = float(np.sqrt(np.mean(resid**2) + 1e-8))

        models[col] = (beta, sigma, other_cols)

    return models

def log_norm_pdf(x, mu, sigma):
    return -0.5 * np.log(2 * np.pi * sigma**2) - 0.5 * ((x - mu) ** 2) / (sigma**2)

def mh_sample_scalar(mu, sigma, x0, n_steps=250, burn=120, proposal_sd=0.8, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    x = x0
    kept = []
    accepts = 0

    for t in range(n_steps):
        x_prop = x + rng.normal(0.0, proposal_sd)

        logp_curr = log_norm_pdf(x, mu, sigma)
        logp_prop = log_norm_pdf(x_prop, mu, sigma)

        if np.log(rng.uniform()) < (logp_prop - logp_curr):
            x = x_prop
            accepts += 1

        if t >= burn:
            kept.append(x)

    return float(np.mean(kept)), accepts / n_steps

def mh_impute_dataset(X_miss: pd.DataFrame, models, n_steps=250, burn=120, proposal_scale=0.8, seed=42):
    rng = np.random.default_rng(seed)
    X_imp = X_miss.copy()

    for col in X_imp.columns:
        X_imp[col] = X_imp[col].fillna(X_imp[col].mean())

    acc = {col: [] for col in X_imp.columns}

    for col, (beta, sigma, other_cols) in models.items():
        miss_idx = X_miss[col].isna()
        if not miss_idx.any():
            continue

        proposal_sd = proposal_scale * sigma

        for i in X_miss.index[miss_idx]:
            x_other = X_imp.loc[i, other_cols].to_numpy()
            mu = float(np.r_[1.0, x_other] @ beta)

            x0 = float(X_imp.loc[i, col])
            x_mean, a = mh_sample_scalar(mu, sigma, x0, n_steps=n_steps, burn=burn, proposal_sd=proposal_sd, rng=rng)
            X_imp.loc[i, col] = x_mean
            acc[col].append(a)

    acc_summary = {col: (np.mean(v) if len(v) else np.nan) for col, v in acc.items()}
    return X_imp, acc_summary


In [ ]:
mh_results = []
mh_errors = {}
mh_acc = {}

for r in missing_rates:
    pack = data_by_rate[r]
    models = fit_conditional_models(pack["X_train_miss"])

    Xtr_imp, acc_tr = mh_impute_dataset(pack["X_train_miss"], models, n_steps=250, burn=120, proposal_scale=0.8, seed=42)
    Xte_imp, acc_te = mh_impute_dataset(pack["X_test_miss"],  models, n_steps=250, burn=120, proposal_scale=0.8, seed=99)

    met = evaluate_logreg(Xtr_imp, pack["y_train"], Xte_imp, pack["y_test"])
    mh_results.append({"MissingRate": int(r * 100), **met})

    err_te = imputation_error(pack["X_test_true"], Xte_imp, pack["mask_test"])
    mh_errors[r] = {"test": err_te}
    mh_acc[r] = {"train": acc_tr, "test": acc_te}

mh_results_df = pd.DataFrame(mh_results).set_index("MissingRate").sort_index()
display(mh_results_df)

print("MH acceptance rates (mean per feature):")
for r in missing_rates:
    print(f"\nMissing {int(r*100)}%")
    display(pd.DataFrame({"train_accept": mh_acc[r]["train"], "test_accept": mh_acc[r]["test"]}))




## 8) Gibbs sampling imputation (from scratch)

We use a Bayesian linear-Gaussian conditional model with weak priors and Gibbs updates:
- sample regression parameters given current completed data
- sample missing entries given parameters
We return the post-burn-in mean for missing entries.


In [ ]:
def invgamma_sample(a, b, rng):
    return 1.0 / rng.gamma(shape=a, scale=1.0 / b)

def gibbs_impute_dataset(
    X_miss: pd.DataFrame,
    n_iter=250,
    burn=120,
    tau2=10.0,
    a0=2.0,
    b0=2.0,
    seed=42
):
    rng = np.random.default_rng(seed)
    X = X_miss.copy()

    for col in X.columns:
        X[col] = X[col].fillna(X[col].mean())

    cols = list(X.columns)
    missing_mask = {col: X_miss[col].isna().to_numpy() for col in cols}

    sums = {col: np.zeros(len(X), dtype=float) for col in cols}
    kept = 0

    for it in range(n_iter):
        for col in cols:
            other_cols = [c for c in cols if c != col]

            y = X[col].to_numpy()
            X_other = X[other_cols].to_numpy()
            X_design = np.c_[np.ones(len(X_other)), X_other]

            p = X_design.shape[1]
            XtX = X_design.T @ X_design
            Xty = X_design.T @ y

            V_inv = XtX + (1.0 / tau2) * np.eye(p)
            V = np.linalg.inv(V_inv)
            m = V @ Xty

            resid = y - X_design @ m
            a_n = a0 + len(y) / 2.0
            b_n = b0 + 0.5 * (resid @ resid)

            sigma2 = invgamma_sample(a_n, b_n, rng)
            beta = rng.multivariate_normal(m, sigma2 * V)

            miss = missing_mask[col]
            if np.any(miss):
                mu_all = X_design @ beta
                y_new = y.copy()
                y_new[miss] = rng.normal(loc=mu_all[miss], scale=np.sqrt(sigma2), size=miss.sum())
                X[col] = y_new

        if it >= burn:
            kept += 1
            for col in cols:
                miss = missing_mask[col]
                if np.any(miss):
                    sums[col][miss] += X[col].to_numpy()[miss]

    X_out = X.copy()
    for col in cols:
        miss = missing_mask[col]
        if np.any(miss) and kept > 0:
            vals = X_out[col].to_numpy()
            vals[miss] = sums[col][miss] / kept
            X_out[col] = vals

    return X_out




In [ ]:
gibbs_results = []
gibbs_errors = {}

for r in missing_rates:
    pack = data_by_rate[r]

    Xtr_imp = gibbs_impute_dataset(pack["X_train_miss"], n_iter=250, burn=120, tau2=10.0, seed=42)
    Xte_imp = gibbs_impute_dataset(pack["X_test_miss"],  n_iter=250, burn=120, tau2=10.0, seed=99)

    met = evaluate_logreg(Xtr_imp, pack["y_train"], Xte_imp, pack["y_test"])
    gibbs_results.append({"MissingRate": int(r * 100), **met})

    err_te = imputation_error(pack["X_test_true"], Xte_imp, pack["mask_test"])
    gibbs_errors[r] = {"test": err_te}

gibbs_results_df = pd.DataFrame(gibbs_results).set_index("MissingRate").sort_index()
display(gibbs_results_df)




## 9) Clean report tables and plots (all missingness levels)

We summarize:
- AUC / LogLoss / F1 vs missingness
- Average RMSE / MAE vs missingness (test masked entries)
- Compact final summary table (AUC + AvgRMSE)


In [ ]:
def prep_perf(df_perf, method_name):
    out = df_perf.copy()
    out["Method"] = method_name
    out = out.reset_index()
    out = out.rename(columns={out.columns[0]: "MissingRate"})
    return out[["Method", "MissingRate", "Accuracy", "AUC", "F1", "LogLoss"]]

perf = pd.concat([
    prep_perf(mean_results_df, "Mean"),
    prep_perf(mice_results_df, "MICE"),
    prep_perf(mh_results_df, "Metropolis-Hastings"),
    prep_perf(gibbs_results_df, "Gibbs")
], ignore_index=True).sort_values(["MissingRate", "Method"]).reset_index(drop=True)

display(perf)

auc_pivot = perf.pivot(index="MissingRate", columns="Method", values="AUC").sort_index()
ll_pivot  = perf.pivot(index="MissingRate", columns="Method", values="LogLoss").sort_index()
f1_pivot  = perf.pivot(index="MissingRate", columns="Method", values="F1").sort_index()

print("AUC")
display(auc_pivot)

print("LogLoss")
display(ll_pivot)

print("F1")
display(f1_pivot)


In [ ]:
def avg_error(err_dict, rates, metric="RMSE"):
    rows = []
    for r in rates:
        rows.append({"MissingRate": int(r*100), metric: float(err_dict[r]["test"][metric].mean())})
    return pd.DataFrame(rows).set_index("MissingRate").sort_index()

avg_rmse = pd.concat([
    avg_error(mean_errors, missing_rates, "RMSE").rename(columns={"RMSE":"Mean"}),
    avg_error(mice_impute_errors, missing_rates, "RMSE").rename(columns={"RMSE":"MICE"}),
    avg_error(mh_errors, missing_rates, "RMSE").rename(columns={"RMSE":"Metropolis-Hastings"}),
    avg_error(gibbs_errors, missing_rates, "RMSE").rename(columns={"RMSE":"Gibbs"})
], axis=1)

avg_mae = pd.concat([
    avg_error(mean_errors, missing_rates, "MAE").rename(columns={"MAE":"Mean"}),
    avg_error(mice_impute_errors, missing_rates, "MAE").rename(columns={"MAE":"MICE"}),
    avg_error(mh_errors, missing_rates, "MAE").rename(columns={"MAE":"Metropolis-Hastings"}),
    avg_error(gibbs_errors, missing_rates, "MAE").rename(columns={"MAE":"Gibbs"})
], axis=1)

print("Average RMSE (test masked entries)")
display(avg_rmse)

print("Average MAE (test masked entries)")
display(avg_mae)


In [ ]:
def line_plot(pivot_df, title, ylab):
    plt.figure(figsize=(7, 4))
    for col in pivot_df.columns:
        plt.plot(pivot_df.index, pivot_df[col], marker="o", label=col)
    plt.title(title)
    plt.xlabel("Missingness (%)")
    plt.ylabel(ylab)
    plt.xticks(pivot_df.index)
    plt.legend()
    plt.tight_layout()
    plt.show()

line_plot(auc_pivot, "AUC vs Missingness", "AUC")
line_plot(ll_pivot,  "LogLoss vs Missingness", "LogLoss")
line_plot(f1_pivot,  "F1 vs Missingness", "F1")
line_plot(avg_rmse,  "Average RMSE vs Missingness (test masked entries)", "Average RMSE")


In [ ]:
final_summary = pd.concat([
    auc_pivot.add_prefix("AUC_"),
    avg_rmse.add_prefix("AvgRMSE_")
], axis=1)

display(final_summary)


## 10) Sensitivity analysis (repeat over seeds)

We repeat the full workflow over a small set of random seeds and report mean ± std of:
- AUC
- Avg RMSE (test masked entries)


In [ ]:
SEEDS = [11, 22, 33, 44, 55]  # increase if time allows

sens_rows = []

for seed in SEEDS:
    X_s2, y_s2 = stratified_sample(df, y_full, continuous_cols, N=1000, seed=seed)

    X_tr2, X_te2, y_tr2, y_te2 = train_test_split(
        X_s2, y_s2, test_size=0.25, random_state=seed, stratify=y_s2
    )

    for r in missing_rates:
        seed_r = int(r * 10_000) + seed
        mask_tr = make_mcar_mask(X_tr2, r, seed=seed_r)
        mask_te = make_mcar_mask(X_te2, r, seed=seed_r + 1)

        Xtr_m = apply_mask(X_tr2, mask_tr)
        Xte_m = apply_mask(X_te2, mask_te)

        # Mean
        Xtr_mean, Xte_mean = mean_impute(Xtr_m, Xte_m)
        auc_mean = evaluate_logreg(Xtr_mean, y_tr2, Xte_mean, y_te2)["AUC"]
        rmse_mean = float(imputation_error(X_te2, Xte_mean, mask_te)["RMSE"].mean())

        # MICE
        imp = IterativeImputer(random_state=seed, sample_posterior=True, max_iter=15, initial_strategy="mean")
        Xtr_mice = pd.DataFrame(imp.fit_transform(Xtr_m), columns=Xtr_m.columns, index=Xtr_m.index)
        Xte_mice = pd.DataFrame(imp.transform(Xte_m), columns=Xte_m.columns, index=Xte_m.index)
        auc_mice = evaluate_logreg(Xtr_mice, y_tr2, Xte_mice, y_te2)["AUC"]
        rmse_mice = float(imputation_error(X_te2, Xte_mice, mask_te)["RMSE"].mean())

        # MH
        models = fit_conditional_models(Xtr_m)
        Xtr_mh, _ = mh_impute_dataset(Xtr_m, models, n_steps=220, burn=90, proposal_scale=0.8, seed=seed)
        Xte_mh, _ = mh_impute_dataset(Xte_m, models, n_steps=220, burn=90, proposal_scale=0.8, seed=seed+100)
        auc_mh = evaluate_logreg(Xtr_mh, y_tr2, Xte_mh, y_te2)["AUC"]
        rmse_mh = float(imputation_error(X_te2, Xte_mh, mask_te)["RMSE"].mean())

        # Gibbs
        Xtr_g = gibbs_impute_dataset(Xtr_m, n_iter=220, burn=90, tau2=10.0, seed=seed)
        Xte_g = gibbs_impute_dataset(Xte_m, n_iter=220, burn=90, tau2=10.0, seed=seed+200)
        auc_g = evaluate_logreg(Xtr_g, y_tr2, Xte_g, y_te2)["AUC"]
        rmse_g = float(imputation_error(X_te2, Xte_g, mask_te)["RMSE"].mean())

        sens_rows += [
            {"Seed": seed, "MissingRate": int(r*100), "Method": "Mean", "AUC": auc_mean, "AvgRMSE": rmse_mean},
            {"Seed": seed, "MissingRate": int(r*100), "Method": "MICE", "AUC": auc_mice, "AvgRMSE": rmse_mice},
            {"Seed": seed, "MissingRate": int(r*100), "Method": "Metropolis-Hastings", "AUC": auc_mh, "AvgRMSE": rmse_mh},
            {"Seed": seed, "MissingRate": int(r*100), "Method": "Gibbs", "AUC": auc_g, "AvgRMSE": rmse_g},
        ]

sens_df = pd.DataFrame(sens_rows)

sens_summary = (
    sens_df.groupby(["MissingRate", "Method"])
    .agg(
        AUC_mean=("AUC", "mean"),
        AUC_std=("AUC", "std"),
        RMSE_mean=("AvgRMSE", "mean"),
        RMSE_std=("AvgRMSE", "std")
    )
    .reset_index()
    .sort_values(["MissingRate", "Method"])
)

display(sens_summary)


In [ ]:
def plot_mean_std(summary_df, mean_col, std_col, title, ylab):
    plt.figure(figsize=(7,4))
    for method in summary_df["Method"].unique():
        sub = summary_df[summary_df["Method"] == method].sort_values("MissingRate")
        x = sub["MissingRate"].to_numpy()
        y = sub[mean_col].to_numpy()
        yerr = sub[std_col].to_numpy()
        plt.errorbar(x, y, yerr=yerr, marker="o", capsize=4, label=method)
    plt.title(title)
    plt.xlabel("Missingness (%)")
    plt.ylabel(ylab)
    plt.xticks(sorted(summary_df["MissingRate"].unique()))
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_mean_std(sens_summary, "AUC_mean", "AUC_std", "Sensitivity: AUC (mean ± std)", "AUC")
plot_mean_std(sens_summary, "RMSE_mean", "RMSE_std", "Sensitivity: Avg RMSE (mean ± std)", "Average RMSE")


## 11) Lightweight convergence checks (MCMC)

We show:
- MH trace + running mean for one missing entry (10% case)
- Gibbs trace + running mean for the same conditional (10% case)
This supports basic convergence/mixing discussion.


In [ ]:
def running_mean(x):
    x = np.asarray(x, dtype=float)
    return np.cumsum(x) / (np.arange(len(x)) + 1)

r = 0.10
pack = data_by_rate[r]
Xtr_m = pack["X_train_miss"].copy()
mask_tr = pack["mask_train"]

col = "age"
miss_idx = Xtr_m.index[mask_tr[col]]
if len(miss_idx) == 0:
    raise ValueError("No missing entries found to monitor. Try another column.")

i0 = miss_idx[0]
print("Monitoring:", col, "row:", i0)

models = fit_conditional_models(Xtr_m)
beta, sigma, other_cols = models[col]

X_init = Xtr_m.copy()
for c in X_init.columns:
    X_init[c] = X_init[c].fillna(X_init[c].mean())

x_other = X_init.loc[i0, other_cols].to_numpy()
mu = float(np.r_[1.0, x_other] @ beta)

rng = np.random.default_rng(123)
x = float(X_init.loc[i0, col])
proposal_sd = 0.8 * sigma

mh_chain = []
accepts = 0
n_steps = 600
burn = 200

for t in range(n_steps):
    x_prop = x + rng.normal(0.0, proposal_sd)
    if np.log(rng.uniform()) < (log_norm_pdf(x_prop, mu, sigma) - log_norm_pdf(x, mu, sigma)):
        x = x_prop
        accepts += 1
    mh_chain.append(x)

plt.figure(figsize=(7,3))
plt.plot(mh_chain, alpha=0.8)
plt.axvline(burn, linestyle="--")
plt.plot(running_mean(mh_chain), linewidth=2)
plt.title(f"MH trace + running mean (accept rate ~ {accepts/n_steps:.2f})")
plt.xlabel("Iteration")
plt.ylabel(col)
plt.tight_layout()
plt.show()

rng = np.random.default_rng(456)
gibbs_chain = []
sigma2 = sigma**2
x = float(X_init.loc[i0, col])

for t in range(n_steps):
    x = rng.normal(loc=mu, scale=np.sqrt(sigma2))
    gibbs_chain.append(x)

plt.figure(figsize=(7,3))
plt.plot(gibbs_chain, alpha=0.8)
plt.axvline(burn, linestyle="--")
plt.plot(running_mean(gibbs_chain), linewidth=2)
plt.title("Gibbs trace + running mean (single-entry conditional)")
plt.xlabel("Iteration")
plt.ylabel(col)
plt.tight_layout()
plt.show()
